# Cube Packcage
The custom cube package is preinstalled.  just load the library and start to use it 
<hr/>


### Load Library and Login to CubeAPI

In [22]:
# load all library, it is already installed
library(devtools)
library(httr)
library(jsonlite)
library(stringr)
library(rapportools)
library(htmlwidgets)
library(rlist)
library(data.table)
library(cloudml)
library(readr)
library(logger)
library(cube)
setwd("~/")
log_threshold(INFO)

In [24]:
# create a CubeAPI object
cube_api = CubeAPI$new()

INFO [2021-01-06 13:49:37] Version: 0.0.2
INFO [2021-01-06 13:49:37] http://10.105.16.22/metadata-service/


In [ ]:
# only need to login once per month
# after run, click on "verification_uri_complete" to finish verification
cube_api$login()

### 1. Get Metadata Definition

In [ ]:
# retrieve metadata element
response = cube_api$get_element()

In [ ]:
str(response)

In [ ]:
# get the content of the response
content = content(response)

In [ ]:
str(content)

In [ ]:
# to data frame
data.frame(do.call(rbind, content$results))[c("id", "element_label", "element_collection_label", "element_abbreviation")]

### 2. Get Bucket data from metadata collection

In [25]:
storage_info = cube_api$get_metadata_collection_storage_info()

INFO [2021-01-06 13:49:42] GET: http://10.105.16.22/metadata-service/metadata_repository/collection/
INFO [2021-01-06 13:49:43] status_code: 200
INFO [2021-01-06 13:49:43] number of accession_ids: 24
INFO [2021-01-06 13:49:43] GET: http://10.105.16.22/metadata-service/metadata_repository/element_instance/
INFO [2021-01-06 13:49:44] status_code: 200
INFO [2021-01-06 13:49:44] number of row returned: 24


In [26]:
storage_info

,accession_id,uri,bucket_name,file_name
,<chr>,<chr>,<chr>,<chr>
1,JAXDS0000O,gs://jax-cube-prd-ctrl-01-project-data/20200607_20-cube-001-run5,gs://jax-cube-prd-ctrl-01-project-data,20200607_20-cube-001-run5
2,JAXDS0000N,gs://jax-cube-prd-ctrl-01-project-data/20200515-20-cube-001-run4,gs://jax-cube-prd-ctrl-01-project-data,20200515-20-cube-001-run4
3,JAXDS0000M,gs://jax-cube-prd-ctrl-01-project-data/20200316_20-cube-001-run2,gs://jax-cube-prd-ctrl-01-project-data,20200316_20-cube-001-run2
4,JAXDS0000L,gs://jax-cube-prd-ctrl-01-project-data/20200207_19-churchill-002-run3,gs://jax-cube-prd-ctrl-01-project-data,20200207_19-churchill-002-run3
5,JAXDS0000K,gs://jax-cube-prd-ctrl-01-project-data/20200203_19-churchill-002-run2,gs://jax-cube-prd-ctrl-01-project-data,20200203_19-churchill-002-run2
6,JAXDS0000J,gs://jax-cube-prd-ctrl-01-project-data/20191231_19-churchill-002,gs://jax-cube-prd-ctrl-01-project-data,20191231_19-churchill-002
7,JAXDS0000I,gs://jax-cube-prd-ctrl-01-project-data/20200128_19-churchill-004-run3,gs://jax-cube-prd-ctrl-01-project-data,20200128_19-churchill-004-run3
8,JAXDS0000H,gs://jax-cube-prd-ctrl-01-project-data/20200102_19-churchill-004-run2,gs://jax-cube-prd-ctrl-01-project-data,20200102_19-churchill-004-run2
9,JAXDS0000G,gs://jax-cube-prd-ctrl-01-project-data/20191223_19-churchill-004,gs://jax-cube-prd-ctrl-01-project-data,20191223_19-churchill-004


In [30]:
storage_info[1, 3]

[1] "gs://jax-cube-prd-ctrl-01-project-data"

In [27]:
## get bucket data
bucket_data = get_bucket_data(bucket_name = storage_info[1, 3], file_name = storage_info[1, 4] )
bucket_data

Warning message in file(file, "rt"):
“cannot open file 'gs/jax-cube-prd-ctrl-01-project-data/20200607_20-cube-001-run5': No such file or directory”


ERROR: Error in file(file, "rt"): cannot open the connection


### 3. Get Metadata Definition with a Parameter

In [ ]:
response = cube_api$get_element(element_id = 85)

In [ ]:
str(content(response))

### 4. Get Metadata with accession ids 

In [ ]:
response = cube_api$get_element_instance(element_id = 85, accession_ids = c("JAXAS00031", "JAXAS00032", "JAXAS00034"))

In [ ]:
str(content(response)$results)

### 5. Get Metadata Collection

In [ ]:
response = cube_api$get_metadata_collection()

In [ ]:
json = response_json_to_data(response)

In [ ]:
str(json$results)

In [ ]:
accession_ids = json$results[,2][[1]]$accession_id
accession_ids

In [ ]:
response = cube_api$get_element_instance(accession_ids = c(accession_ids))

In [ ]:
results = content(response)$results
results[[1]][[1]]

In [ ]:
data_url = results[[1]][[1]][[1]][[4]]
url_parts = str_split(data_url, "/")
bucket_name = paste0("gs://", url_parts[[1]][4], "/", url_parts[[1]][5])
file_name =  url_parts[[1]][6]

# load the data from the GCP bucket
data_dir = gs_data_dir( bucket_name )

# read the file into a dataframe
df <- read.table(file.path(data_dir, file_name), sep = '\t',header = TRUE)
df

### 5. Call Any Get endpoint
Call any get end point.  All the endpoints are listed here: https://metadata-service-sqa.azurewebsites.net/swagger/

In [ ]:
# specify a end point
response = cube_api$get_end_point("metadata_definition/property/?element_id=85")

In [ ]:
content(response)$count

In [ ]:
# another end point
response = cube_api$get_end_point("/metadata_repository/collection_item/1/")

In [ ]:
content(response)

### 6. Post endpoint

In [ ]:
response = cube_api$post_element_instance_filter(element_id = 122, page = 1, page_size = 2)

In [ ]:
str(content(response))

### 7. More Information

In [ ]:
?CubeAPI

In [ ]:
?Cube